## TASK 3

This code has been written to implement a function which returns operator for any unitary targeting any qubit(s) for any circuit size - therefore an implementation of the U3 gate. 

For this reason, the input format is slightly different from the one mentioned in the task description notebook. The input format for this simulator is:

my_circuit = \[{ "total_qubits":2, "theta":[np.pi/2], "phi": [0], "lambda": [np.pi],"control_qubits":[], "target_qubits": [0] },

{ "total_qubits":2, "theta":[0, np.pi], "phi": [0, 0], "lambda": [0, np.pi], "control_qubits":[0], "target_qubits": [1] },

...
\]

The control_qubits term can be passed as an empty list if for example implementing a single-qubit gate.

The Theta, phi and lambda lists should contain $2^{**}l$ elements, l being the number of control qubits. This is becuase for this generalised simulator, a unitary can be implemented on the target qubit for each permutation of conditions on the control qubits as given in the additional info document shown below:

<a href="https://ibb.co/3f4kDk2"><img src="https://i.ibb.co/jVbHcHm/Capture.png" alt="Capture" border="0"></a>



In [224]:
import numpy as np
from numpy.random import choice

In [225]:
def get_ground_state(num_qubits):
    # return vector of size 2**num_qubits with all zeroes except first element which is 1
    ground_state = np.zeros((2**num_qubits,1))
    ground_state[0] = 1
    return ground_state

def unitary(theta, phi, lambd):
    gate_unitary = [[np.cos(theta/2), -np.exp(1j * lambd) * np.sin(theta / 2)],
                    [np.exp(1j * phi) * np.sin(theta / 2), np.exp(1j * lambd + 1j * phi) * np.cos(theta / 2)]]
    return gate_unitary


def get_operator(total_qubits, theta, phi, lambd, control_qubits, target_qubits):
    # return unitary operator of size 2**n x 2**n for given gate and target qubits
    l = len(control_qubits)
    m = len(target_qubits)
    n = total_qubits

    operator = 0
    # Define 2x2 Identity

    I = np.identity(2)

    # Define projection operator |0><0|

    P0x0 = np.array([[1, 0],
                     [0, 0]])

    # Define projection operator |1><1|

    P1x1 = np.array([[0, 0],
                     [0, 1]])

    projections = [P0x0, P1x1]

    for i in range(2**l):
        
        #here binary indexes have been used to decide the permutations on control qubits
        
        bin_array = np.array(list(np.binary_repr(i).zfill(l))).astype(np.int8)
        control_index = 0
        target_index = 0
        
        #multi-qubit gate with control terms
        if(l!=0):
            
            #deciding the first qubit term
            if(control_qubits[control_index]==0):
                term = projections[bin_array[control_index]]
                control_index+=1

            elif(target_qubits[target_index]==0):
                term = unitary(theta[i], phi[i], lambd[i])

            else:
                term = I
                
            #operations for rest of the qubits(tensor product in succession to the first qubit)
            for j in range(1,n):
                if(j in control_qubits):
                    term = np.kron(term, projections[bin_array[control_index]])
                    control_index+=1

                elif(j in target_qubits):
                    term = np.kron(term,unitary(theta[i], phi[i], lambd[i]))

                else:
                    term = np.kron(term, I)
            operator += term
        
        #for gates without control terms
        else:
            if target_qubits[0] == 0: 
                operator = unitary(theta[i], phi[i], lambd[i])
            else:
                operator = I        
                
            for j in range(1,n):
                if (j in target_qubits):
                    operator = np.kron(operator, unitary(theta[i], phi[i], lambd[i]))
                else:
                    operator = np.kron(operator, I)
    
    return operator

def run_program(initial_state, program):
    # read program, and for each gate:
    #   - calculate matrix operator
    #   - multiply state with operator
    # return final state
    final_state = initial_state
    for p in program:
        operator = get_operator(p["total_qubits"], p["theta"], p["phi"],
                            p["lambda"], p["control_qubits"], p["target_qubits"])
        final_state = np.dot(operator, final_state)/np.linalg.norm(np.dot(operator, final_state))
    return final_state

def measure_all(state_vector, total_qubits):
    # choose element from state_vector using weighted random and return it's index
    tobin = np.vectorize(np.binary_repr)
    list_of_candidates = tobin(np.arange(len(state_vector)))
    list_of_candidates = [item.zfill(total_qubits) for item in list_of_candidates]   
    abs_vec = np.vectorize(abs)
    draw = choice(list_of_candidates, 1, 
                  p=np.ndarray.flatten(abs_vec(state_vector)**2))
    return draw

def get_counts(total_qubits, state_vector, num_shots):
    # simply execute measure_all in a loop num_shots times and
    # return object with statistics in following form:
    #   {
    #      element_index: number_of_ocurrences,
    #      element_index: number_of_ocurrences,
    #      element_index: number_of_ocurrences,
    #      ...
    #   }
    statistic = {}
    for i in range(num_shots):
        
        value = measure_all(state_vector, total_qubits)
        if(value[0] in statistic):
            statistic[value[0]] +=1
        else:
            statistic[value[0]] =1
    # (only for elements which occoured - returned from measure_all)
    return statistic

### Example Implementation: 

2 Qubit Simulator as given in the task document:

In [226]:
#define total_qubits:

total_qubits = 2
# Define program:

my_circuit = [
{ "total_qubits":2, "theta":[np.pi/2], "phi": [0], "lambda": [np.pi],"control_qubits":[], "target_qubits": [0] }, 
{ "total_qubits":2, "theta":[0,np.pi], "phi": [0,0], "lambda":[0,np.pi], "control_qubits":[0], "target_qubits": [1] }
]


# Create "quantum computer" with 2 qubits (this is actually just a vector :) )

my_qpu = get_ground_state(2)


# Run circuit

final_state = run_program(my_qpu, my_circuit)


# Read results

counts = get_counts(total_qubits, final_state, 1000)

print(counts)


{'00': 502, '11': 498}


### Example Implementation: 

8 Qubit Simulator for the same gates as before:

In [227]:
#define total_qubits:

total_qubits = 8

# Define program:

my_circuit = [
{ "total_qubits":total_qubits, "theta":[np.pi/2], "phi": [0], "lambda": [np.pi],"control_qubits":[], "target_qubits": [0] }, 
{ "total_qubits":total_qubits, "theta":[0,np.pi], "phi": [0,0], "lambda":[0,np.pi], "control_qubits":[0], "target_qubits": [1] }
]


# Create "quantum computer" with 2 qubits (this is actually just a vector :) )

my_qpu = get_ground_state(total_qubits)


# Run circuit

final_state = run_program(my_qpu, my_circuit)


# Read results

counts = get_counts(total_qubits, final_state, 1000)

print(counts)


{'11000000': 479, '00000000': 521}


### Example Implementation: 

4 Qubit Simulator h-gate then CCX(0,1,2):

In [228]:
#define total_qubits:

total_qubits = 4

# Define program:

my_circuit = [
    { "total_qubits":total_qubits, "theta":[np.pi/2], "phi": [0], "lambda": [np.pi],"control_qubits":[], "target_qubits": [0] }, 
    { "total_qubits":total_qubits, "theta":[0,0,0,np.pi], "phi": [0,0,0,0], "lambda":[0,0,0,np.pi], "control_qubits":[0,1], "target_qubits": [2] }
]


# Create "quantum computer" with 2 qubits (this is actually just a vector :) )

my_qpu = get_ground_state(total_qubits)


# Run circuit

final_state = run_program(my_qpu, my_circuit)


# Read results

counts = get_counts(total_qubits, final_state, 1000)

print(counts)


{'1000': 513, '0000': 487}
